In [1]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
help(mp_drawing.DrawingSpec)

Help on class DrawingSpec in module mediapipe.python.solutions.drawing_utils:

class DrawingSpec(builtins.object)
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |  
 |  DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -------------------------------------------------

In [4]:
from IPython.display import Image

Image(url="https://i.imgur.com/3j8BPdc.png", width=300)

In [5]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.56125814
y: 0.53053725
z: -1.3016666
visibility: 0.9997309
, x: 0.59696805
y: 0.46652126
z: -1.2362516
visibility: 0.9993181
, x: 0.61905354
y: 0.47329208
z: -1.236522
visibility: 0.99943155
, x: 0.6390972
y: 0.48084328
z: -1.2366822
visibility: 0.9992113
, x: 0.5406807
y: 0.45712444
z: -1.2273774
visibility: 0.99937755
, x: 0.5239642
y: 0.4567784
z: -1.2263561
visibility: 0.9995053
, x: 0.51025033
y: 0.45727193
z: -1.2268765
visibility: 0.9994068
, x: 0.6729216
y: 0.5235709
z: -0.75701475
visibility: 0.999288
, x: 0.49229437
y: 0.4955753
z: -0.6888343
visibility: 0.9996131
, x: 0.59843445
y: 0.6096252
z: -1.1180891
visibility: 0.99974495
, x: 0.52322257
y: 0.6040735
z: -1.1008759
visibility: 0.99979526
, x: 0.8308929
y: 0.9000876
z: -0.4476952
visibility: 0.9966679
, x: 0.297057
y: 0.86629146
z: -0.38261956
visibility: 0.9986078
, x: 0.9741564
y: 1.266703
z: -0.5340203
visibility: 0.080532886
, x: 0.08933002
y: 1.2565383
z: -0.43808523
visibility: 0.4531608
, x: 0.96506196
y: 1.

In [6]:
len(landmarks)

33

In [7]:
for lndmrk in mp_pose.PoseLandmark: 
    print(lndmrk)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [8]:
[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

[0.2780523896217346, 0.9009314179420471]

In [9]:
[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

[0.09386811405420303, 1.3339899778366089]

In [10]:
[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

[0.11718781292438507, 1.712680459022522]

In [11]:
def calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist):
    right_shoulder = np.array(right_shoulder)  # Shoulder landmark
    right_elbow = np.array(right_elbow)  # Elbow landmark
    right_wrist = np.array(right_wrist)  # Wrist landmark
    
    # Calculate the angle between the vectors formed by the landmarks
    angle = np.degrees(np.arccos(np.dot(right_shoulder - right_elbow, right_wrist - right_elbow) / (np.linalg.norm(right_shoulder - right_elbow) * np.linalg.norm(right_wrist - right_elbow))))
    
    return angle


In [12]:
right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


In [13]:
right_shoulder, right_elbow, right_wrist

([0.2780523896217346, 0.9009314179420471],
 [0.09386811405420303, 1.3339899778366089],
 [0.11718781292438507, 1.712680459022522])

In [14]:
calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)

153.43563455115654

In [15]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(75, 822)

Start of Bench Press form logic


In [16]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.093868114
y: 1.33399
z: -0.4262677
visibility: 0.43173063

In [17]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.2780524
y: 0.9009314
z: -0.37807876
visibility: 0.998755

In [18]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

x: 0.38037205
y: 1.7965255
z: 0.11045297
visibility: 0.00045101662

In [19]:
def calculate_form_angle(elbow, shoulder, hip):
    elbow = np.array(elbow)  # Right elbow landmark
    shoulder = np.array(shoulder)  # Right shoulder landmark
    hip = np.array(hip)      # Right hip landmark
    
    
    radians = np.arctan2(shoulder[1]-hip[1], shoulder[0]-hip[0]) - np.arctan2(elbow[1]-hip[1], elbow[0]-hip[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle


In [20]:
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]


In [21]:
elbow, shoulder, hip

([0.09386811405420303, 1.3339899778366089],
 [0.2780523896217346, 0.9009314179420471],
 [0.3803720474243164, 1.796525478363037])

In [22]:
calculate_form_angle(elbow, shoulder, hip)

25.257261779571678

In [23]:
tuple(np.multiply(right_shoulder, [640, 480]).astype(int))
tuple(np.multiply(right_elbow, [640, 480]).astype(int))
tuple(np.multiply(right_wrist, [640, 480]).astype(int))

(75, 822)

In [24]:
cap = cv2.VideoCapture(0)

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)

            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Overhead Press Checker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [37]:
cap = cv2.VideoCapture(0)

# OHP counter variables
counter = 0 
stage = None
feedback = ""

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for ohp landmarks
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            
            
            # Calculate ohp angle
            overhead_angle = calculate_overhead_press_angle(right_shoulder, right_elbow, right_wrist)
            form_angle = calculate_form_angle(elbow, shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(overhead_angle), 
                           tuple(np.multiply(right_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(form_angle), 
                           tuple(np.multiply(shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Overhead Press Counter Algorithm
            if overhead_angle < 70:
                stage = "down" 

            if stage == "down" and overhead_angle >= 30 and form_angle < 21:
                    feedback = "Tuck elbows in"
            elif stage == "down" and overhead_angle <= 30 and form_angle > 21: 
                    feedback = "Good form"

                    
            if stage == "down" and overhead_angle <= 40 and form_angle < 20:
                feedback = "Good bottom form"
            elif stage == "down" and overhead_angle >= 40 and form_angle > 20:
                 feedback = "Tuck elbows inside"

            if overhead_angle > 150 and stage =="down":
                stage="up"
                if form_angle < 21:
                    feedback = "DW"
                counter += 1
            
                       
        except:
            pass
        
        # Render Overhead Press Counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
         # Text for 'SQUATS'
        cv2.putText(image, 'BENCH PRESS', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'STAGE'
        cv2.putText(image, 'STAGE', (165, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (160, 80), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        # Text for 'FORM' (Ensure it fits within the box)
        feedback_lines = feedback.split('\n')
        if len(feedback_lines) > 0:
            feedback_line1 = feedback_lines[0]
        else:
            feedback_line1 = ''

        if len(feedback_lines) > 1:
            feedback_line2 = feedback_lines[1]
        else:
            feedback_line2 = ''

        cv2.putText(image, 'FORM', (290, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, feedback_line1, (270, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, feedback_line2, (270, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Bench Press Tracker', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    
    cv2.destroyAllWindows()
